# Explore CREG025.L75 experiments to assess the AW subduction process 
> ## This script allows to compute Heat flux Time series through 3 sections:
>> ### Extreme North of the LargeFraWSC box
>> ### Fram Strait within LargeFraWSC box
>> ### Southern strait of the LargeFraWSC box

In [53]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pylab as plt

%matplotlib inline

In [54]:
s_year=1980 ; e_year=2015
RYEAR=str(s_year)+'-'+str(e_year)

In [55]:
CONFIG='CREG025.L75'   ; CASE=''
CONFCASE=CONFIG

In [56]:
# List of GS± experiments names:
case_Ref='BVHNTMXP'   #; case_Ref='BCTGE27NTMX'  ;  case_Ref='BVHNTMXD'

##### Read CREG025.L75 grid information 

In [57]:
main_dir='/net/5lpo154/export/drakkar-h3/drakkar/CONFIGS/'
grid_dir=main_dir+CONFIG+'/GRID/'

# Mask of the grid
ds_mask=xr.open_dataset(grid_dir+CONFCASE+'_byte_mask.nc')
ds_mask

# Horizontal mesh
ds_mesh_hgr=xr.open_dataset(grid_dir+CONFCASE+'_mesh_hgr.nc')
ds_mesh_hgr

# Vertical mesh
ds_mesh_zgr=xr.open_dataset(grid_dir+CONFCASE+'_mesh_zgr.nc')
ds_mesh_zgr

<xarray.Dataset>
Dimensions:       (t: 1, x: 528, y: 603, z: 75)
Dimensions without coordinates: t, x, y, z
Data variables:
    nav_lon       (y, x) float32 ...
    nav_lat       (y, x) float32 ...
    nav_lev       (z) float32 ...
    time_counter  (t) float64 ...
    mbathy        (t, y, x) int16 ...
    misf          (t, y, x) int16 ...
    isfdraft      (t, y, x) float32 ...
    e3t_0         (t, z, y, x) float64 ...
    e3u_0         (t, z, y, x) float64 ...
    e3v_0         (t, z, y, x) float64 ...
    e3w_0         (t, z, y, x) float64 ...
    hdept         (t, y, x) float32 ...
    hdepw         (t, y, x) float32 ...
    gdept_1d      (t, z) float64 ...
    gdepw_1d      (t, z) float64 ...
    e3t_1d        (t, z) float64 ...
    e3w_1d        (t, z) float64 ...
Attributes:
    file_name:  mesh_zgr.nc
    TimeStamp:  08/08/2016 13:31:30 +0200

In [58]:
ds_grid=xr.Dataset()
ds_grid
#ds_grid['tmask2D']=(('time','z','y','x'),ds_mask['tmask'])
ds_grid['Tarea']=ds_mask['tmask'][0,0,:,:]*ds_mesh_hgr['e1t'][0,:,:]*ds_mesh_hgr['e2t'][0,:,:]
ds_grid['Farea']=ds_mask['fmask'][0,0,:,:]*ds_mesh_hgr['e1f'][0,:,:]*ds_mesh_hgr['e2f'][0,:,:]
ds_grid['tmask2D']=ds_mask['tmask'].sel(z=0).squeeze()
ds_grid.coords['gphif']=(('y','x'),ds_mesh_hgr['gphif'][0,:,:])
ds_grid.coords['glamf']=(('y','x'),ds_mesh_hgr['glamf'][0,:,:])
ds_grid.coords['gphit']=(('y','x'),ds_mesh_hgr['gphit'][0,:,:])
ds_grid.coords['glamt']=(('y','x'),ds_mesh_hgr['glamt'][0,:,:])


ds_grid['e1v3D'] = (('z','y','x'),np.tile(ds_mesh_hgr['e1v'][0,:,:].squeeze(),(75,1,1)))
ds_grid['e2u3D'] = (('z','y','x'),np.tile(ds_mesh_hgr['e2u'][0,:,:].squeeze(),(75,1,1)))

ds_grid['e1ve3v3D'] = (('z','y','x'),ds_grid['e1v3D']*ds_mesh_zgr['e3v_0'].squeeze())
ds_grid['e2ue3u3D'] = (('z','y','x'),ds_grid['e2u3D']*ds_mesh_zgr['e3u_0'].squeeze())

ds_grid
#ds_grid['NS_Tarea']=np.tile(e1t,(e3t.shape[0],1,1))


<xarray.Dataset>
Dimensions:   (x: 528, y: 603, z: 75)
Coordinates:
    gphif     (y, x) float32 25.4716 25.471653 25.471708 ... 55.095528 55.02518
    glamf     (y, x) float32 -93.62534 -93.37534 ... 102.25591 102.051285
    gphit     (y, x) float32 25.359825 25.359877 ... 55.021885 54.95175
    glamt     (y, x) float32 -93.75031 -93.50031 ... 102.193756 101.988945
Dimensions without coordinates: x, y, z
Data variables:
    Tarea     (y, x) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    Farea     (y, x) float64 0.0 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0 0.0
    tmask2D   (y, x) int8 ...
    e1v3D     (z, y, x) float64 2.51e+04 2.51e+04 2.51e+04 ... 1.52e+04 1.52e+04
    e2u3D     (z, y, x) float64 2.487e+04 2.487e+04 ... 9.997e+03 9.943e+03
    e1ve3v3D  (z, y, x) float64 2.569e+04 2.569e+04 ... 3.104e+06 3.105e+06
    e2ue3u3D  (z, y, x) float64 2.545e+04 2.545e+04 ... 2.042e+06 2.031e+06

## Read the time series data 

In [59]:
!date

Wed Jan 16 21:18:13 UTC 2019


### Compute the ∆Q heat flux looses between the 3 meridional sections of the LargeFraWSC box 

In [60]:
# Ref experiment data
DATA_PATH='/home/ctalandi/TOOLS/SSHFS/drakkarcomC/'+CONFIG+'/'+'CREG025.L75-'+case_Ref+'-MEAN/1m/'+RYEAR+'/'
ds_RefgT=xr.open_mfdataset(DATA_PATH+'CREG025.L75-'+case_Ref+'_y*m*.1m_gridT.nc',autoclose=True)

In [61]:
DATA_PATH='/home/ctalandi/TOOLS/SSHFS/drakkarcomC/'+CONFIG+'/'+'CREG025.L75-'+case_Ref+'-MEAN/1m/'+RYEAR+'/'
ds_RefgU=xr.open_mfdataset(DATA_PATH+'CREG025.L75-'+case_Ref+'_y*m*.1m_gridU.nc',autoclose=True)

IOError: no files to open

In [ ]:
!date

## Select the North_South section within the LargeFraWSC box

In [ ]:
ds_diagsSecZY=xr.Dataset()  
# Extreme North section of the Large FraWSC box 
SX_jloc_s=307 ; SX_jloc_e=346 ; SX_iloc_s=322  ;  SX_iloc_e=322
  
# Select the WSC northern section temperaure and velocitie
ds_diagsSecZY['Ref_XS_FraWSC_T']= (('time_counter','z','y'),ds_RefgT['votemper'][:,:,SX_jloc_s:SX_jloc_e+1,SX_iloc_s])
ds_diagsSecZY['Ref_XS_FraWSC_U']= (('time_counter','z','y'),ds_RefgU['vozocrtx'][:,:,SX_jloc_s:SX_jloc_e+1,SX_iloc_s])

# Mask temperature using a 0°C temperature criteria 
Tcrit=0.
ds_diagsSecZY['Ref_XS_FraWSC_Tmsk']=xr.where(ds_diagsSecZY['Ref_XS_FraWSC_T']<Tcrit,0.,ds_diagsSecZY['Ref_XS_FraWSC_T'])
ds_diagsSecZY['Ref_XS_FraWSC_Umsk']=xr.where(ds_diagsSecZY['Ref_XS_FraWSC_T']<Tcrit,0.,ds_diagsSecZY['Ref_XS_FraWSC_U'])

# Compute the time-mean field
ds_diagsSecZY['Ref_XS_FraWSC_meanT']= (('z','y'),ds_diagsSecZY['Ref_XS_FraWSC_T'].mean(dim='time_counter'))
ds_diagsSecZY['Ref_XS_FraWSC_meanU']= (('z','y'),ds_diagsSecZY['Ref_XS_FraWSC_U'].mean(dim='time_counter'))
ds_diagsSecZY['Ref_XS_FraWSC_meanTmsk']=xr.where(ds_diagsSecZY['Ref_XS_FraWSC_meanT']<0.,np.nan,ds_diagsSecZY['Ref_XS_FraWSC_meanT'])

# Select the vertical surface of the Northern section
ds_diagsSecZY['Ref_XS_FraWSC_e2e3U']= (('z','y'),ds_grid['e2ue3u3D'][:,SX_jloc_s:SX_jloc_e+1,SX_iloc_s])


In [ ]:
!date

In [ ]:
# Control the location of the mask
plt.figure(figsize=(20,10))
plt.subplot(211)
plt.pcolormesh(ds_diagsSecZY['Ref_XS_FraWSC_meanT'][::-1,:],vmin=-5., vmax=5.,cmap='seismic')
CS1=plt.contour(ds_diagsSecZY['Ref_XS_FraWSC_meanT'][::-1,:],levels=[0.,2., 3.],colors='k',origin='lower',linestyles='solid')
plt.title('Mean Temperature along the EXTREME NORTH section')
plt.ylabel('Depth levels')
plt.xlabel('Along section')
plt.clabel(CS1, inline=True, fmt='%1.0f')


plt.subplot(212)
C=plt.pcolormesh(ds_diagsSecZY['Ref_XS_FraWSC_meanTmsk'][::-1,:],vmin=-5., vmax=5.,cmap='seismic')
CS1=plt.contour(ds_diagsSecZY['Ref_XS_FraWSC_meanTmsk'][::-1,:],levels=[0.,2., 3.],colors='k',origin='lower',linestyles='solid')
plt.title('Mean Temperature along the EXTREME NORTH section')
plt.ylabel('Depth levels')
plt.xlabel('Along section')
plt.clabel(CS1, inline=True, fmt='%1.0f')
cbar = plt.colorbar(C,format='%3.0f',orientation='horizontal',shrink=0.8, extend='both')
cbar.set_label(r'[DegC]',fontsize=18)


plt.savefig('./'+case_Ref+'_y'+RYEAR+'_TimeMean_Temp_LargeFraWSC_ZYSec.pdf')

In [ ]:
!date

### Compute the TIME-MEAN heat transport through the sections

In [ ]:
# Compute the heat transport through the sections
rho_swater=1027. # [J.kg.K-1] 
Cp=4160. # [J.kg.K-1]


ds_diagsSecZY['Ref_XS_FraWSC_meanHflx']=rho_swater*Cp*ds_diagsSecZY['Ref_XS_FraWSC_e2e3U']* \
                                ds_diagsSecZY['Ref_XS_FraWSC_meanTmsk']*ds_diagsSecZY['Ref_XS_FraWSC_meanU']


In [ ]:
!date

### Check the way the calculation is done for the TIME-MEAN heat flux 

In [ ]:

plt.figure(figsize=(20,10))
plt.subplot(211)
plt.pcolormesh(ds_diagsSecZY['Ref_XS_FraWSC_meanU'][::-1,:]*1e2,vmin=-10., vmax=10.,cmap='seismic')
CS1=plt.contour(ds_diagsSecZY['Ref_XS_FraWSC_meanU'][::-1,:]*1e2,levels=[0.,2., 3.],colors='k',origin='lower',linestyles='solid')
plt.title('Mean Velocities along the EXTREME NORTH section')
plt.ylabel('Depth levels')
plt.xlabel('Along section')
plt.clabel(CS1, inline=True, fmt='%1.0f')

plt.subplot(212)
m_scal=1e-9
C=plt.pcolormesh(ds_diagsSecZY['Ref_XS_FraWSC_meanHflx'][::-1,:]*m_scal,vmin=-100,vmax=100,cmap='seismic')
CS1=plt.contour(ds_diagsSecZY['Ref_XS_FraWSC_meanHflx'][::-1,:]*m_scal,levels=np.arange(-100.,100.,25.),colors='k',origin='lower',linestyles='solid')
plt.title('Mean Heat hflx along the EXTREME NORTH section (GW)')
plt.ylabel('Depth levels')
plt.xlabel('Along section')
plt.clabel(CS1, inline=True, fmt='%3.0f')
cbar = plt.colorbar(C,format='%3.0f',orientation='horizontal',shrink=0.8, extend='both')
cbar.set_label(r'[GW]',fontsize=18)

plt.savefig('./'+case_Ref+'_y'+RYEAR+'_TimeMean_VHflx_LargeFraWSC_ZYSec.pdf')

In [ ]:
!date

### Compute & output the heat flux Time-series through the sections

In [ ]:
# Compute the heat transport through the sections
rho_swater=1027. # [J.kg.K-1] 
Cp=4160. # [J.kg.K-1]

ds_diagsSecZY['Ref_XS_FraWSC_Hflx']=rho_swater*Cp*ds_diagsSecZY['Ref_XS_FraWSC_e2e3U']* \
                                ds_diagsSecZY['Ref_XS_FraWSC_Tmsk']*ds_diagsSecZY['Ref_XS_FraWSC_U']

ds_diagsSecZY['Ref_XS_FraWSC_Hflx_TiSe']= ds_diagsSecZY['Ref_XS_FraWSC_Hflx'].sum(dim=('z','y'))

# Compute the volume through the 3 sections based on the T criteria defined above
ds_diagsSecZY['Ref_XS_FraWSC_Vol']=ds_diagsSecZY['Ref_XS_FraWSC_e2e3U']*ds_diagsSecZY['Ref_XS_FraWSC_Umsk']
ds_diagsSecZY['Ref_XS_FraWSC_Vol_TiSe']= ds_diagsSecZY['Ref_XS_FraWSC_Vol'].sum(dim=('z','y'))


ds_diagsSecZY.to_netcdf('./'+case_Ref+'_y'+RYEAR+'_LargeFraWSC_TiSe_ZYSec.nc', unlimited_dims={'time':True})

In [ ]:
!date

In [ ]:
ds_diagsSecZY

In [ ]:
monthly_time

### Plot the heat flux Time-series through sections

In [ ]:

monthly_time=pd.date_range(start=str(s_year)+'-01-01',end=str(e_year)+'-12-31',freq='M')

m_alpha=1e-12
plt.figure(figsize=(20,15))

ax=plt.subplot(514)   ; 
ax.plot(monthly_time,(ds_diagsSecZY['Ref_XS_FraWSC_Hflx_TiSe'])*m_alpha, 'k')

ax.plot(monthly_time,np.zeros(len(monthly_time))*m_alpha,'k')
ax.set_xlim(str(s_year),str(e_year+1))
ax.set_ylim(-120.,00.)
plt.ylabel('Net Heat flux (TW) ')
plt.title(r'Monthly mean ZYSec of Large FraWSC box')
plt.grid(True)

plt.savefig('./'+case_Ref+'_y'+RYEAR+'_TiSe_Hflx_LargeFraWSC_ZYSec.pdf')

In [ ]:
!date